# GCN(graph convolution network)
 - graph structure
   - vertex(node) 와 edge로 구성 
 - Node Feature Matrix , Adjacency Matrix
   - NFM : node의 feature정보를 가지는 $matrix_{f*n}$(f: feature 수,n : node 수)
   - AM  : node들간에 edge의 정보(ex 인접성)를 나타내는 $matrix_{n*n}$
 
 - make each layer of network update values of each node feature
 - 즉, Hidden state(node feature matrix)를 업데이트한다.
 - 수식: $H_i^{(l+1)} = act(\sum_{j \in N(i)}H_j^l * W^l + b^l)$ ...(1)
 - $j \in N(i)$는 $node_i$와 연결되어 있는 노드들이다.
 - 위의 식에서 보았듯이 같은 weight를 사용(weight sharing)
 - 하지만 실제로 구현을 할때 모든 node를 찾아봐 연결되어 있는 node들을 찾는것은 불가능 하다.
 - 따라서 위에서 정의한 Ajacency matrix를 사용할 것이다.
 - (1) = $H^{(l+1)} = act(AH^l*W^l + b^l)$으로 표현가능하다.(A = AM)
 
 - Readout layer : Permutation lnvariance
   - 노드의 순서(그래프의 모양(ex 위아래로 뒤집힌))가 달라도 graph는 같은 모양이고 의미하는 바도 같지만 matrix의 의미는 바뀌므로 이를 방지 하기 위한 layer
   - ex) $Z_G = act(\sum_{i \in G}MLP(H_i^L) ) $

 - 전반적인 구조
   - Graph input($G(X,A)$) -> Graph conv layer -> ... -> Readout layer -> Predictor -> Labels

 - 즉, GCN은 (1)번식의 H를 업데이트해가면서 작동을 한다.(1 epoch)
 - 또한 위의 과정을 걸쳐 나온 predictor과 ture값을 비교하여 loss를 계산하여 weight를 update해 가면서 epoch를 진행한다.


# Advanced Techniques of GCN
- cnn의 inception module과 같은 방식
- Adjancency Matrix를 한번더 곱하여 거리가 2인 node들도 고려
- 구조(하나의 input graph에 대해)
  - graph conv -> graph conv  -> grap conv ....(1)
  - graph conv -> graph conv  ........(2)
  - graph conv .......(3)
  - (1),(2), (3)을 concatenation을 한다.

- resnet의 방식
  - $H_{i,sc}^{(l+1)} = H_i^{l+1} + H_i(l)$

- attention 방식
 - seq2seq의 문제점 : 항상 같은 길이의 vector를 반환
 - 

# Model Construction
Graph Convolution Network, 즉 GCN을 pytorch를 이용하여 구현하여봅시다. 이를 위해 다음과 같은 sub module들을 구현
 - GCNLayer: node feature matrix와 adjacency matrix의 list를 받아 graph convolution 연산을 수행하는 module 입니다.
 - (Gated)SkipConnection: ResNet에서 사용되었던 skip connection technique을 구현한 module 입니다.
 - GCNBlock: node feature matrix와 adjacency matrix의 list를 받아 원하는 갯수의 GCNLayer를 통과시킨 후, (gated)skip connection을 적용하는 module 입니다.
 - ReadOut: graph structrure에 permutation invariance를 주기 위하여 linear layer를 거친 뒤 batch 별로 summation하는 module 입니다.
 - Predictor: ReadOut layer로부터의 graph feature vector로부터 logP value를 예측하기 위한 linear layer module 입니다.